In [ ]:
'''
Data munging libraries

'''
import random

import numpy as np
import pandas as pd
import statsmodels.api as sm

import joblib
'''
Visualization Libraries

'''
import seaborn as sns
%matplotlib inline
#%matplotlib notebook
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 2) 
from bokeh.plotting import figure,  show, gridplot
from bokeh.io import output_notebook
from bokeh.layouts import row, column

'''
ML libraries

'''

from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score;
from sklearn.model_selection import  train_test_split ;
from sklearn.pipeline import Pipeline;

# Define file name of model_data¶

In [ ]:
file_name_model_data = "../model_data/stg_model_data_new.gzip"

In [ ]:
model_df=pd.read_csv(file_name_model_data, compression='gzip' ,encoding="ISO-8859-2")

In [ ]:
model_df.describe()

In [ ]:
model_df.info()

# Steam Turbine Model

In [ ]:
mw1='GT3 Generator Watts Max Selected {Avg}'
mw2='GT4 Generator Watts Max Selected {Avg}'
# std1 = 'GT3 Generator Watts Max Selected {StdDev}'
# std2 = 'GT4 Generator Watts Max Selected {StdDev}'
hpflow1 = 'HRSG 3 HP STEAM FLOW {KPPH}'
crhflow1 = "HRSG 3 COLD REHEAT STEAM FLOW DUALSEL {KPPH}"
ipflow1 = "HRSG 3 DUALSEL IP STEAM FLOW {KPPH}"
lpflow1 = 'HRSG 3 LP STEAM FLOW {KPPH}'
dbfuel1 = 'HRSG3_DB_MMBTU'

hpflow2 = 'HRSG 4 HP STEAM FLOW {KPPH}'
crhflow2 = "HRSG 4 COLD REHEAT STEAM FLOW DUALSEL {KPPH}"
ipflow2 = "HRSG 4 DUALSEL IP STEAM FLOW {KPPH}"
lpflow2 = 'HRSG 4 LP STEAM FLOW {KPPH}'
dbfuel2 = 'HRSG4_DB_MMBTU'
#ipflow2 = 'HRSG 9 COLD REHEAT STEAM FLOW DUALSEL {Avg}'


stgmw = "STG gross mw {Avg}"
#stgstd = "STG gross mw {StdDev}"
backpress = "Exhaust Vacuum {Avg}"
#bpstd = "Exhaust Vacuum {StdDev}"

gt3amb= "GT3 Inlet Duct Temperature {Avg}"
gt4amb= "GT4 Inlet Duct Temperature {Avg}"
watertemp = "CND CIRC WTR INLET TEMPERATURE {Avg}"
baro='SITE AMBIENT CONDITIONS BARO PRESS XMTR {Avg}'
rh='SITE AMBIENT CONDITIONS REL HUMIDITY {Avg}'
amb='Dry Bulb Temp {Avg}'


In [ ]:
mydf = model_df[(model_df[watertemp]>0)&
                (model_df[stgmw]>45)
               ].reset_index(drop=True)
#waterdf = mydf[[drybulb, rh, watertemp, backpress1,backpress2,backpress3,hpsteam1,hpsteam2,hrhflow1,hrhflow2,baro]]

In [ ]:
model_df.columns

In [ ]:
mydf['amb']=mydf[amb]#(mydf[gt3amb]+mydf[gt4amb])/2

In [ ]:
gt1on='GT3_on'
gt2on='GT4_on'
mydf[gt1on]=mydf[mw1].apply(lambda x: 1 if x>5 else 0)
mydf[gt2on]=mydf[mw2].apply(lambda x: 1 if x>5 else 0)

In [ ]:
mydf['HRH 3 Flow']=mydf[crhflow1]+mydf[ipflow1]
mydf['HRH 4 Flow']=mydf[crhflow2]+mydf[ipflow2]

In [ ]:
hrhflow1='HRH 3 Flow'
hrhflow2='HRH 4 Flow'

In [ ]:
waterdf = mydf

In [ ]:
waterdf['HP_flow']=waterdf[hpflow1]*waterdf[gt1on]+waterdf[hpflow2]*waterdf[gt2on]
waterdf['LP_flow']=waterdf[lpflow1]*waterdf[gt1on]+waterdf[lpflow2]*waterdf[gt2on]
waterdf['HRH_flow']=(waterdf[crhflow1]+waterdf[ipflow1])*waterdf[gt1on]+(waterdf[crhflow2]+waterdf[ipflow2])*waterdf[gt2on]

In [ ]:
def model_pca(pca_frac=None,layers=(40,40),es=True,n_iter=200,tol=0.0001,patience=10,random=2301):
    ppl=Pipeline([('scaler',StandardScaler()),('pca',PCA(n_components=pca_frac,random_state=random)),
                  ('estimator',MLPRegressor(hidden_layer_sizes=layers,
                                            early_stopping=es,
                                            tol=tol,max_iter=n_iter,random_state=random,n_iter_no_change=patience))
                 ])
    return ppl

In [ ]:
def model(layers=(40,40),es=True,n_iter=200,tol=0.0001,patience=20):
    ppl=Pipeline([('scaler',StandardScaler()),
                  ('estimator',MLPRegressor(hidden_layer_sizes=layers,
                                            early_stopping=es,tol=tol,max_iter=n_iter,
                                            random_state=2301,n_iter_no_change=patience))
                 ])
    return ppl

In [ ]:
def model_lm_pca(pca_frac=None,random=2301):
    ppl=Pipeline([('scaler',StandardScaler()),('pca',PCA(n_components=pca_frac,random_state=random)),
                  ('estimator',LinearRegression())
                 ])
    return ppl

In [ ]:
def model_poly_pca(pca_frac=None,random=2301):
    ppl=Pipeline([('scaler',StandardScaler()),('pca',PCA(n_components=pca_frac,random_state=random)),
                  ('quadratic',PolynomialFeatures()),
                  ('estimator',LinearRegression())
                 ])
    return ppl

In [ ]:
## backpressure model
x =waterdf[['HP_flow','amb',rh,baro]]
y = waterdf[backpress]
#bp_rf = RandomForestRegressor(max_depth=20,n_estimators=300, random_state=6567)
#bp_ = model_pca(layers=(200,200,200),n_iter=500)
bp_ = model_poly_pca()
bp_=bp_.fit(x,y)


In [ ]:
# residual plot
pred = bp_.predict(x)
err = y - pred

output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="Residual Plot", x_axis_label='Index', 
            y_axis_label='Error')
t1.scatter(list(range(len(err))), err)
show(t1)
print(np.sqrt(np.var(err)))
print(np.mean(np.abs((y - pred) / y)))

In [ ]:
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred)
show(t1)

In [ ]:
waterdf['bperr']=abs(err)

In [ ]:
## backpressure model
x =waterdf[waterdf['bperr']<0.5][['HP_flow','amb',rh,baro]]
y = waterdf[waterdf['bperr']<0.5][backpress]
#bp_rf = RandomForestRegressor(max_depth=20,n_estimators=300, random_state=6567)
#bp_ = model_pca(layers=(200,200,200),n_iter=500)
bp_ = model_poly_pca()
bp_=bp_.fit(x,y)

In [ ]:
pred = bp_.predict(x)
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred)
show(t1)

In [ ]:
total_hp_flow=waterdf['HP_flow']
total_hrh_flow=waterdf['HRH_flow']
total_lp_flow=waterdf['LP_flow']
back_press = waterdf[backpress]
waterdf['hp_hrh_lp_flow']=total_hp_flow+total_hrh_flow+total_lp_flow

In [ ]:
#newflow=total_hp_flow+total_hrh_flow
#newpress=wa_hp_press+wa_hrh_press

In [ ]:
#x = pd.concat([pd.Series(total_hp_flow,name = 'total_hp_flow'),
#               pd.Series(wa_hp_press, name = 'wa_hp_press'),
#               pd.Series(wa_hp_temp, name = 'wa_hp_temp'),              
#               pd.Series(total_hrh_flow,name = 'total_hrh_flow'),
#               pd.Series(wa_hrh_press, name = 'wa_hrh_press'),
#               pd.Series(wa_hrh_temp, name = 'wa_hrh_temp'),
               
#               pd.Series(back_press, name = 'backpressure')],axis=1)
#y = waterdf[stgmw]

In [ ]:
# x = pd.concat([pd.Series(total_hp_flow,name = 'total_hp_flow'),
#               pd.Series(wa_hp_press, name = 'wa_hp_press'),                          
#               pd.Series(total_hrh_flow,name = 'total_hrh_flow'),
#               pd.Series(wa_hrh_press, name = 'wa_hrh_press'),                              
#               pd.Series(back_press, name = 'backpressure')],axis=1)
# y = waterdf[stgmw]

In [ ]:
stg_1x1df=waterdf[((waterdf[gt1on]+waterdf[gt2on])<2)&(waterdf[stgmw]>55)].reset_index(drop=True)
stg_2x1df=waterdf[(waterdf[gt1on]+waterdf[gt2on])==2].reset_index(drop=True)

In [ ]:
def addmw(row,add1x1=4,flow=450,scalef=1):
    if row[1]>= 600:
        mw=row[0]-add1x1*scalef
    elif row[1]<400:
        mw=row[0]
    else:
        mw=row[0]+add1x1*flow/row[1]
    return mw

# Only run the below line if 1x1 is problematic

In [ ]:
#stg_1x1df['stgmw_mod']=stg_1x1df[[stgmw,'HP_flow']].apply(lambda x: addmw(x,scalef=1.5),axis=1)

In [ ]:
#stg_1x1df[[stgmw,'stgmw_mod']]

In [ ]:
x=stg_1x1df[['HP_flow','amb',rh,baro]]
y = stg_1x1df[stgmw]

In [ ]:
#stg1x1_=model(layers=(200,200,200),n_iter=500)
stg1x1_=model_poly_pca()
stg1x1_=stg1x1_.fit(x,y)

In [ ]:
# residual plot
pred1 = stg1x1_.predict(x)
err1 = y - pred1


output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="Residual Plot", x_axis_label='Index', 
            y_axis_label='Error')
t1.scatter(list(range(len(err1))), err1)
show(t1)


print(np.sqrt(np.var(err1)))
print(np.mean(np.abs((y - pred1) / y)))



In [ ]:
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred1)
show(t1)

In [ ]:
# Remove outliers mw ~ cit
ransac_1x1mw = RANSACRegressor(model_poly_pca(),
                         max_trials=100, 
                         min_samples=50, 
                         loss='absolute_loss', 
                         residual_threshold=2, 
                         random_state=2301)

ransac_1x1mw=ransac_1x1mw.fit(stg_1x1df[['HP_flow','amb',rh,baro]], stg_1x1df[stgmw])

mw_pred=ransac_1x1mw.predict(stg_1x1df[['HP_flow','amb',rh,baro]])

inlier_mask = ransac_1x1mw.inlier_mask_

stg_1x1df=stg_1x1df.assign(normal_mw = lambda im: inlier_mask)
stg_1x1df['mw_pred']=mw_pred

sns.lmplot(x='mw_pred', y=stgmw, data=stg_1x1df,hue="normal_mw", height=8,fit_reg=True,scatter_kws={"s": 25},
           line_kws={"color":"black","linewidth":4},ci=None)

In [ ]:
stg_1x1dffiltr=stg_1x1df#.loc[5500:,].reset_index(drop=True)

In [ ]:
# Remove outliers mw ~ cit
ransac_1x1mw = RANSACRegressor(model_poly_pca(),
                         max_trials=100, 
                         min_samples=50, 
                         loss='absolute_loss', 
                         residual_threshold=3, 
                         random_state=2301)

ransac_1x1mw=ransac_1x1mw.fit(stg_1x1dffiltr[['HP_flow','amb',rh,baro]], stg_1x1dffiltr[stgmw])

mw_pred=ransac_1x1mw.predict(stg_1x1dffiltr[['HP_flow','amb',rh,baro]])

inlier_mask = ransac_1x1mw.inlier_mask_

stg_1x1dffiltr=stg_1x1dffiltr.assign(normal_mw = lambda im: inlier_mask)

sns.lmplot(x='HP_flow', y=stgmw, data=stg_1x1dffiltr,hue="normal_mw", height=8,fit_reg=True,scatter_kws={"s": 25},
           line_kws={"color":"black","linewidth":4},ci=None)

In [ ]:
stg_1x1dfclean = stg_1x1dffiltr[stg_1x1dffiltr["normal_mw"]].reset_index(drop=True)

In [ ]:
x=stg_1x1dfclean[['HP_flow','amb',rh,baro]]
y = stg_1x1dfclean[stgmw]

In [ ]:
#stg1x1_=model(layers=(200,200,200),n_iter=500)
stg1x1_=model_poly_pca()
stg1x1_=stg1x1_.fit(x,y)

In [ ]:
# residual plot
pred1 = stg1x1_.predict(x)
err1 = y - pred1


output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="Residual Plot", x_axis_label='Index', 
            y_axis_label='Error')
t1.scatter(list(range(len(err1))), err1)
show(t1)


print(np.sqrt(np.var(err1)))
print(np.mean(np.abs((y - pred1) / y)))



In [ ]:
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred1)
show(t1)

In [ ]:
# Remove outliers mw ~ cit
ransac_mw = RANSACRegressor(model_poly_pca(),
                         max_trials=100, 
                         min_samples=50, 
                         loss='absolute_loss', 
                         residual_threshold=5, 
                         random_state=2301)

ransac_mw=ransac_mw.fit(stg_2x1df[['HP_flow','amb',rh,baro]], stg_2x1df[stgmw])

mw_pred=ransac_mw.predict(stg_2x1df[['HP_flow','amb',rh,baro]])

inlier_mask = ransac_mw.inlier_mask_

stg_2x1df=stg_2x1df.assign(normal_mw = lambda im: inlier_mask)

stg_2x1df['mw_pred']=mw_pred

sns.lmplot(x='mw_pred', y=stgmw, data=stg_2x1df,hue="normal_mw", height=8,fit_reg=True,scatter_kws={"s": 25},
           line_kws={"color":"black","linewidth":4},ci=None)


In [ ]:
stg_2x1dfclean = stg_2x1df[stg_2x1df["normal_mw"]].reset_index(drop=True)

In [ ]:
x=stg_2x1dfclean[['HP_flow','amb',rh,baro]]
y = stg_2x1dfclean[stgmw]

In [ ]:
stg2x1_=model_poly_pca()
#stg2x1_=model_pca(layers=(200,200,200),n_iter=1000,patience=80)
stg2x1_=stg2x1_.fit(x,y)

In [ ]:
# residual plot
pred1 = stg2x1_.predict(x)
err1 = y - pred1


output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="Residual Plot", x_axis_label='Index', 
            y_axis_label='Error')
t1.scatter(list(range(len(err1))), err1)
show(t1)


print(np.sqrt(np.var(err1)))
print(np.mean(np.abs((y - pred1) / y)))


In [ ]:
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred1)
show(t1)

# The three cells below are for tuning stg at min load

In [ ]:
# stg_2x1dffiltr1=stg_2x1dfclean.loc[7000:10000,:].reset_index(drop=True)
# stg_2x1dffiltr2=stg_2x1dfclean.loc[11500:,:].reset_index(drop=True)
# stg2x1low=stg_2x1dfclean[stg_2x1dfclean[stgmw]<150].reset_index(drop=True)
# stg_2x1dfnew=pd.concat([stg_2x1dffiltr1,stg_2x1dffiltr1,stg2x1low])

In [ ]:
# def addmw(row,add2x1=7,flow=800,scalef=1):
#     if row[1]>= flow:
#         mw=row[0]
#     #elif row[1]<flow:
#     else:
#         mw=row[0]-add2x1*scalef
#     #else:
#     #    mw=row[0]+add1x1*flow/row[1]
#     return mw

In [ ]:
# stg_2x1dfnew['stgmw_mod']=stg_2x1dfnew[[stgmw,'HP_flow']].apply(lambda x: addmw(x),axis=1)

In [ ]:
# x=stg_2x1dfnew[['HP_flow','amb',rh,baro]]
# y = stg_2x1dfnew['stgmw_mod']

In [ ]:
#stg2x1_=model_lm_pca()
# stg2x1_=model_lm_pca()
# stg2x1_=stg2x1_.fit(x,y)

In [ ]:
# residual plot
pred1 = stg2x1_.predict(x)
err1 = y - pred1


output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="Residual Plot", x_axis_label='Index', 
            y_axis_label='Error')
t1.scatter(list(range(len(err1))), err1)
show(t1)


print(np.sqrt(np.var(err1)))
print(np.mean(np.abs((y - pred1) / y)))


In [ ]:
output_notebook()
t1 = figure(plot_width=350, plot_height=350,title="pred vs actual", x_axis_label='actual', 
            y_axis_label='pred')
t1.scatter(y,pred1)
show(t1)

In [ ]:
pkl = "../../../pickles/stg.pkl"

models = {
            'back_pressure<back_press><hp_flow|db|rh|baro>': bp_,            
            'stg1x1<mw><hp_flow|db|rh|baro>': stg1x1_,
            'stg2x1<mw><hp_flow|db|rh|baro>': stg2x1_
            
}

with open(pkl, "wb") as f:
    joblib.dump(models, f)
    print(f'{f.name}')